# 03. Defect Detection - 결함 시각화

이 노트북은 학습된 YOLOv11 모델을 사용하여 테스트 이미지에서 결함을 검출하고 시각화합니다.

## 시각화 과정
1. 환경 설정 및 학습된 모델 로드
2. 테스트 이미지에 대한 추론 수행
3. 검출된 결함 영역을 바운딩 박스로 표시
4. 결과 이미지 저장 및 확인

## 1. 환경 설정

In [ ]:
# Ultralytics YOLO 라이브러리 설치
!pip install ultralytics -q
!pip install opencv-python -q

In [ ]:
import os
import cv2
import glob
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from IPython.display import Image, display
import matplotlib.pyplot as plt

print("✅ 라이브러리 로드 완료")

## 2. Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. 작업 디렉토리 및 모델 경로 설정

**중요**: 
- `WORK_DIR`: 본인의 Google Drive 경로로 수정
- `MODEL_PATH`: 01_train.ipynb에서 학습한 모델 경로 (고정되어 있음)

In [ ]:
# Google Drive 내 작업 디렉토리 경로 설정
WORK_DIR = '/content/drive/MyDrive/2026_AI_Advanced_Study-main/3차시/01_Defect_Detection'

# 학습된 모델 경로 (고정된 경로)
MODEL_PATH = os.path.join(WORK_DIR, 'runs/defect/weights/best.pt')

# 작업 디렉토리로 이동
os.chdir(WORK_DIR)
print(f"현재 작업 디렉토리: {os.getcwd()}")

# 모델 파일 존재 확인
if os.path.exists(MODEL_PATH):
    print(f"✅ 모델 발견: {MODEL_PATH}")
else:
    print(f"❌ 모델을 찾을 수 없습니다: {MODEL_PATH}")
    print("   먼저 01_train.ipynb를 실행하여 모델을 학습하세요.")

## 4. 추론 설정

In [ ]:
# 추론 설정
IMAGE_SIZE = 640
GPU_DEVICE = 0

# 테스트 이미지 디렉토리
TEST_IMAGE_DIR = "./data/images/test"

# 결과 저장 디렉토리
PREDICT_DIR = "runs/defect_predict"
os.makedirs(PREDICT_DIR, exist_ok=True)

# 클래스별 색상 (BGR 형식)
COLORS = {
    0: (0, 0, 255),  # 빨간색 - defect
}

print("=== 추론 설정 ===")
print(f"모델: {MODEL_PATH}")
print(f"이미지 크기: {IMAGE_SIZE}")
print(f"테스트 이미지 경로: {TEST_IMAGE_DIR}")
print(f"결과 저장 경로: {PREDICT_DIR}")

## 5. 테스트 이미지 로드

In [ ]:
# 테스트 이미지 목록 가져오기
test_images = sorted(glob.glob(f"{TEST_IMAGE_DIR}/*.jpg"))
print(f"✅ 테스트 이미지 개수: {len(test_images)}장")

if len(test_images) == 0:
    print("❌ 테스트 이미지를 찾을 수 없습니다. TEST_IMAGE_DIR 경로를 확인하세요.")
else:
    print(f"예시 이미지: {test_images[0]}")

## 6. 학습된 모델 로드

In [ ]:
# 학습된 YOLO 모델 로드
model = YOLO(MODEL_PATH)
print(f"✅ 모델 로드 완료: {MODEL_PATH}")

## 7. 추론 수행

모든 테스트 이미지에 대해 결함 검출을 수행합니다.

In [ ]:
print(f"\n{len(test_images)}개 이미지에 대해 추론을 실행합니다...")

# 배치 추론 실행
results = model.predict(
    test_images,
    imgsz=IMAGE_SIZE,
    device=GPU_DEVICE,
    verbose=False
)

print("✅ 추론 완료!")

## 8. 결과 시각화 및 저장

검출된 결함 영역에 바운딩 박스를 그리고 저장합니다.

In [ ]:
# 각 이미지의 추론 결과 처리
saved_count = 0

for i, result in enumerate(results):
    # 파일명 추출
    filename = Path(test_images[i]).stem
    
    # 원본 이미지 로드
    original_image = cv2.imread(test_images[i])
    
    # 바운딩 박스가 있는 경우에만 처리
    if result.boxes is not None and len(result.boxes) > 0:
        # 원본 이미지에 박스 그리기
        annotated_image = original_image.copy()
        
        # 각 탐지된 객체의 박스 처리
        for j, box in enumerate(result.boxes):
            class_id = int(box.cls.item())
            confidence = float(box.conf.item())
            
            # 색상 선택
            color = COLORS.get(class_id, (0, 255, 0))
            
            # 박스 좌표 추출 (xyxy 형식)
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            
            # 박스 그리기
            cv2.rectangle(annotated_image, (x1, y1), (x2, y2), color, 2)
            
            # 클래스 이름과 신뢰도 표시
            label = f"defect: {confidence:.2f}"
            label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            
            # 라벨 배경 그리기
            cv2.rectangle(annotated_image, 
                         (x1, y1 - label_size[1] - 10),
                         (x1 + label_size[0], y1), 
                         color, -1)
            
            # 라벨 텍스트 그리기
            cv2.putText(annotated_image, label, (x1, y1 - 5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # predict 디렉토리에 이미지 저장
        predict_output_path = f"{PREDICT_DIR}/{filename}_detected.png"
        cv2.imwrite(predict_output_path, annotated_image)
        saved_count += 1
        
        if saved_count <= 5:  # 처음 5개만 로그 출력
            print(f"저장됨: {predict_output_path} (결함 {len(result.boxes)}개 검출)")

print(f"\n✅ 총 {saved_count}개의 이미지가 저장되었습니다.")
print(f"저장 위치: {PREDICT_DIR}")

## 9. 결과 확인

저장된 이미지 중 일부를 확인합니다.

In [ ]:
# 저장된 이미지 목록
saved_images = sorted(glob.glob(f"{PREDICT_DIR}/*_detected.png"))

if len(saved_images) > 0:
    print(f"총 {len(saved_images)}개의 결과 이미지가 저장되었습니다.\n")
    
    # 처음 3개 이미지 표시
    num_display = min(3, len(saved_images))
    
    for i in range(num_display):
        img_path = saved_images[i]
        print(f"=== 결과 이미지 {i+1}/{num_display} ===")
        print(f"파일: {Path(img_path).name}\n")
        
        # 이미지 로드 및 표시 (BGR -> RGB 변환)
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(12, 8))
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.title(f"Defect Detection Result - {Path(img_path).name}")
        plt.tight_layout()
        plt.show()
        print("\n")
else:
    print("결과 이미지를 찾을 수 없습니다.")

## 10. 통계 정보

In [ ]:
# 검출 통계 계산
total_detections = 0
images_with_defects = 0

for result in results:
    if result.boxes is not None and len(result.boxes) > 0:
        images_with_defects += 1
        total_detections += len(result.boxes)

print("=== 검출 통계 ===")
print(f"전체 테스트 이미지 수: {len(test_images)}장")
print(f"결함이 검출된 이미지: {images_with_defects}장")
print(f"총 검출된 결함 개수: {total_detections}개")
if images_with_defects > 0:
    print(f"이미지당 평균 결함 개수: {total_detections/images_with_defects:.2f}개")

## 11. 완료

결함 검출 및 시각화가 완료되었습니다.

### 요약
- 테스트 이미지에 대한 추론 수행 완료
- 검출된 결함 영역을 바운딩 박스로 표시
- 결과 이미지를 `predict/` 디렉토리에 저장

### 활용 방안
- 산업 현장의 품질 관리 자동화
- 실시간 결함 검출 시스템 구축
- 제품 불량률 감소 및 생산성 향상

In [ ]:
print("\n" + "="*50)
print("=== 시각화 완료 ===")
print("="*50)
print(f"\n결과 이미지 저장 위치: {PREDICT_DIR}")
if 'saved_images' in dir():
    print(f"저장된 이미지 개수: {len(saved_images)}장")